## Elevation binning using the NASADEM dataset

In this tutorial, you'll learn how to use different classification methods with [xarray-spatial](https://github.com/makepath/xarray-spatial) to classify the terrain elevation levels of the Grand Canyon.

Geo-spatial data [classification](http://wiki.gis.com/wiki/index.php/Classification) algorithms assign groups of data to categories, or classes, for further processing. Classification is used when grouping data points into classes for different colored areas on a choropleth map, for example. [xarray-spatial](https://github.com/makepath/xarray-spatial) is a raster analysis tool and contains different classification methods.

This tutorial walks you through:
1. Loading and rendering the area of interest data using the Grand Canyon's latitude and longitude.
2. Classifying the data using xarray-spatial's [natural breaks](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.natural_breaks.html), [equal interval](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.equal_interval.html), [quantile](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.quantile.html), and [reclassify](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.reclassify.html) functions.


The data is derived primarily from data captured via the [Shuttle Radar Topography Mission](https://www2.jpl.nasa.gov/srtm/) (SRTM) and is stored on Azure Storage in [cloud-optimized GeoTIFF](https://www.cogeo.org/) format.


### 1. Load the area of interest data

In [ ]:
import xarray as xr
import rioxarray

file_name = 'data/colorado_merge_3arc_resamp.tif'
raster = rioxarray.open_rasterio(file_name).sel(band=1).drop_vars('band')
raster.name = 'Colorado Elevation Raster'

xmin, xmax = raster.x.data.min(), raster.x.data.max()
ymin, ymax = raster.y.data.min(), raster.y.data.max()

xmin, xmax, ymin, ymax

In [ ]:
raster.plot.imshow(figsize=(15, 10));

### 2. Classify elevation data

#### Classify with `natural_breaks()`

Use the [natural breaks](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.natural_breaks.html) function to classify data with the [Jenks natural breaks classification](http://wiki.gis.com/wiki/index.php/Jenks_Natural_Breaks_Classification) method. This method is designed to distribute data into classes according to clusters that form a "natural" group within the data. The algorithm minimizes the average deviation from the class mean while also maximizing the deviation from the means of the other groups. Therefore, it is generally not recommended for data with low variance.

In [ ]:
from datashader.transfer_functions import shade
import matplotlib.pyplot as plt
from xrspatial.classify import natural_breaks

natural_breaks_agg = natural_breaks(raster, num_sample=20000, k=15)

shade(natural_breaks_agg, cmap=plt.get_cmap("terrain"), how="linear")

You can see in the image above that the different elevation levels of the Grand Canyon are now identified by a limited number of distinct colors. Each color represents a range of values within the classified data. For example, the dark blue color areas represent the smallest elevation levels with around 700m of altitude, the yellow color areas represent elevation levels of around 1700m, and the white color areas represent the highest elevations of around 2500m of altitude.

#### Classify with `equal_interval()`

To classify data into sets based on intervals of equal width, use the [equal interval](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.equal_interval.html) function. The [equal interval classification](http://wiki.gis.com/wiki/index.php/Equal_Interval_classification) is useful in cases where you want to emphasize the amount of an attribute value relative to the other values.

In [ ]:
from xrspatial.classify import equal_interval

equal_interval_agg = equal_interval(raster, k=15)

shade(equal_interval_agg, cmap=plt.get_cmap("terrain"), how="linear")

#### Classify with `quantile()`

To classify data based on quantile groups of equal size, use the [quantile](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.quantile.html) function. With [quantile classification](http://wiki.gis.com/wiki/index.php/Quantile), each class contains the same amount of data points. This means that each class is equally represented on the map. However, intervals of uneven sizes can lead to an over-weighting of outliers and other effects.

In [ ]:
from xrspatial.classify import quantile

quantile_agg = quantile(raster, k=15)

shade(quantile_agg, cmap=plt.get_cmap("terrain"), how="linear")

### Use custom bins with `reclassify`

To define your own arbitrary bins to classify data, use the [reclassify](https://xarray-spatial.readthedocs.io/reference/_autosummary/xrspatial.classify.reclassify.html) function. This function is helpful to highlight specific sections of your data, for example. Use `reclassify()` to only visualize elevations greater than 2500m:

In [ ]:
from xrspatial.classify import reclassify

bins = range(150, 5200, 350)
new_vals = [val if val > 2500 else 0 for val in bins]

reclass_agg = reclassify(
    agg=raster,
    bins=bins,
    new_values=new_vals,
)

shade(reclass_agg, cmap=plt.get_cmap("terrain"), how="linear")